In [1]:
import pandas as pd

In [2]:
df_customers = pd.read_csv('capstone_dataset/customers_dataset.csv')

df_customers.head(1)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP


In [3]:
df_orders = pd.read_csv('capstone_dataset/orders_dataset.csv')

df_orders.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00


In [4]:
df_orders_clean = df_orders[df_orders['order_status'] == 'delivered']

In [5]:
df_payments = pd.read_csv('capstone_dataset/order_payments_dataset.csv')

df_payments.head(1)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33


In [6]:
order_totals = df_payments.groupby('order_id')['payment_value'].sum()

order_totals.head(1)

order_id
00010242fe8c5a6d1ba2dd792cb16214    72.19
Name: payment_value, dtype: float64

In [13]:
df_customers_order_merged = pd.merge(df_customers, df_orders_clean, on='customer_id', how='left')

df_customers_order_merged.head(1)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00


In [14]:
df_customer_order_payment_merged = pd.merge(df_customers_order_merged, order_totals, on='order_id', how='left')

df_customer_order_payment_merged.head(1)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,146.87


In [15]:
df_customer_order_payment_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_id                    99441 non-null  object 
 1   customer_unique_id             99441 non-null  object 
 2   customer_zip_code_prefix       99441 non-null  int64  
 3   customer_city                  99441 non-null  object 
 4   customer_state                 99441 non-null  object 
 5   order_id                       96478 non-null  object 
 6   order_status                   96478 non-null  object 
 7   order_purchase_timestamp       96478 non-null  object 
 8   order_approved_at              96464 non-null  object 
 9   order_delivered_carrier_date   96476 non-null  object 
 10  order_delivered_customer_date  96470 non-null  object 
 11  order_estimated_delivery_date  96478 non-null  object 
 12  payment_value                  96477 non-null 

In [16]:
df_customer_order_payment_merged[['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']] = df_customer_order_payment_merged[['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']].apply(pd.to_datetime)

In [17]:
#Define analysis date (max date + 1 day)
analysis_date = df_customer_order_payment_merged['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

In [23]:
#Calculate RFM at customer level
rfm = df_customer_order_payment_merged.groupby('customer_id').agg({
    'order_purchase_timestamp' : lambda x: (analysis_date - x.max()).days,
    'order_id' : 'count',
    'payment_value' : 'sum'
})

rfm.columns = ['Recency', 'Frequency','Monetary']


In [25]:
#Recency: lower days = better score
rfm['R_Score'] = pd.cut(
    rfm['Recency'],
    bins=[-1, 30, 90, 180, 365, float('inf')],
    labels=[5, 4, 3, 2, 1]
)

,Recency(day),Frequency,Monetary
customer_id,,,
